In [23]:
import tkinter as tk
from tkinter import ttk

In [24]:
PARAMS = {
    "CO (L/min)": {"min": 1.0, "max": 12.0, "safe_min": 4.0, "safe_max": 8.0, "step": 0.1, "initial": 5.5},
    "SV (mL/beat)": {"min": 20, "max": 150, "safe_min": 60, "safe_max": 100, "step": 1, "initial": 75},
    "HR (bpm)": {"min": 30, "max": 180, "safe_min": 60, "safe_max": 100, "step": 1, "initial": 80},
}

In [25]:
CLINICAL_WARNINGS = {
    "CO (L/min)": {
        "low": "⚠️ Low Cardiac Output: Risk of organ hypoperfusion, hypotension, shock.",
        "high": "⚠️ High Cardiac Output: Possible sepsis, tachycardia, or volume overload."
    },
    "SV (mL/beat)": {
        "low": "⚠️ Low Stroke Volume: Possible heart failure, hypovolemia, or arrhythmia.",
        "high": "⚠️ High Stroke Volume: May indicate volume overload or excessive contractility."
    },
    "HR (bpm)": {
        "low": "⚠️ Bradycardia: Reduced cardiac output, dizziness, or syncope risk.",
        "high": "⚠️ Tachycardia: High oxygen demand and arrhythmia risk."
    }
}


In [26]:
class CardiacOutputMonitorUI(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Non-invasive Cardiac Output Monitor - Demo UI")
        self.geometry("700x400")
        self.configure(bg="white")

        self.parameters = {}
        self.value_labels = {}
        self.status_labels = {}

        self.create_header()
        self.create_parameter_panel()
        self.create_warning_panel()
        self.update_warning()

    # ---------- HEADER ----------
    def create_header(self):
        header_frame = tk.Frame(self, bg="white")
        header_frame.pack(fill=tk.X, pady=10)

        title_label = tk.Label(
            header_frame,
            text="Non-invasive Cardiac Output Monitor",
            font=("Helvetica", 18, "bold"),
            bg="white"
        )
        title_label.pack()

        subtitle_label = tk.Label(
            header_frame,
            text="Adjust parameters and observe range-based warnings",
            font=("Helvetica", 11),
            bg="white",
            fg="gray"
        )
        subtitle_label.pack()

    # ---------- PARAMETER PANEL ----------
    def create_parameter_panel(self):
        panel = tk.Frame(self, bg="white")
        panel.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)

        for row, (name, cfg) in enumerate(PARAMS.items()):
            frame = tk.Frame(
                panel,
                bg="white",
                highlightbackground="#e0e0e0",
                highlightthickness=1,
                padx=10,
                pady=5
            )
            frame.grid(row=row, column=0, sticky="ew", pady=5)
            frame.columnconfigure(1, weight=1)

            # Parameter name label
            name_label = tk.Label(
                frame,
                text=name,
                font=("Helvetica", 12, "bold"),
                bg="white"
            )
            name_label.grid(row=0, column=0, sticky="w")

            # Current value label
            value_label = tk.Label(
                frame,
                text="",
                font=("Helvetica", 12),
                bg="white"
            )
            value_label.grid(row=0, column=1, sticky="e", padx=10)

            # Status label (Normal / Out of Range)
            status_label = tk.Label(
                frame,
                text="Normal",
                font=("Helvetica", 10, "bold"),
                bg="white",
                fg="green"
            )
            status_label.grid(row=0, column=2, sticky="e", padx=10)

            # Slider / Scale
            res = cfg["step"]
            from_ = cfg["min"]
            to_ = cfg["max"]

            var = tk.DoubleVar(value=cfg["initial"])
            scale = tk.Scale(
                frame,
                variable=var,
                from_=from_,
                to=to_,
                orient=tk.HORIZONTAL,
                length=400,
                resolution=res,
                showvalue=False,
                command=lambda _val, n=name: self.on_param_change(n),
                bg="white",
                highlightthickness=0
            )
            scale.grid(row=1, column=0, columnspan=3, sticky="ew", pady=5)

            # Save references
            self.parameters[name] = var
            self.value_labels[name] = value_label
            self.status_labels[name] = status_label

            # Initialize display
            self.update_param_display(name)

    # ---------- WARNING PANEL ----------
    def create_warning_panel(self):
        self.warning_frame = tk.Frame(self, bg="#f5f5f5", height=60)
        self.warning_frame.pack(fill=tk.X, side=tk.BOTTOM)

        self.warning_label = tk.Label(
            self.warning_frame,
            text="All parameters within safe ranges.",
            font=("Helvetica", 12, "bold"),
            bg="#f5f5f5",
            fg="green",
            justify="left"
        )
        self.warning_label.pack(pady=10)

    # ---------- EVENT: parameter changed ----------
    def on_param_change(self, param_name):
        self.update_param_display(param_name)
        self.update_warning()

    # ---------- UPDATE ONE PARAMETER ----------
    def update_param_display(self, param_name):
        cfg = PARAMS[param_name]
        value = self.parameters[param_name].get()

        # Format value
        if "CO" in param_name:
            text = f"{value:.1f}"
        else:
            text = f"{int(value)}"

        # Update numeric label
        self.value_labels[param_name].config(text=text)

        # Check safe range
        safe_min = cfg["safe_min"]
        safe_max = cfg["safe_max"]

        if safe_min <= value <= safe_max:
            self.status_labels[param_name].config(text="Normal", fg="green")
        else:
            self.status_labels[param_name].config(text="Out of Range", fg="red")

    # ---------- UPDATE GLOBAL WARNING (CLINICAL MESSAGES) ----------
    def update_warning(self):
        messages = []

        for name, cfg in PARAMS.items():
            value = self.parameters[name].get()

            if value < cfg["safe_min"]:
                messages.append(CLINICAL_WARNINGS[name]["low"])
            elif value > cfg["safe_max"]:
                messages.append(CLINICAL_WARNINGS[name]["high"])

        if not messages:
            self.warning_frame.config(bg="#f5f5f5")
            self.warning_label.config(
                text="All parameters within safe ranges.",
                fg="green",
                bg="#f5f5f5",
                justify="left"
            )
        else:
            full_msg = "\n".join(messages)
            self.warning_frame.config(bg="#ffe6e6")
            self.warning_label.config(
                text=full_msg,
                fg="red",
                bg="#ffe6e6",
                justify="left"
            )


In [ ]:
app = CardiacOutputMonitorUI()
app.mainloop()
